# VisionAirport

## Imports + Init

In [1]:
from pyspark import SparkContext
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from pyspark.sql.types import *
from pyspark.sql import *
from scipy import stats
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, desc, asc, lit
from pyspark.sql import functions as F
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
from pyspark.sql import SQLContext
try:
    sc = SparkContext("local").getOrCreate()
except:
    print("SC already exists")
sqlContext = SQLContext(sc)
spark = SparkSession(sc)
plt.style.use('ggplot')
DATADIR = "./data"

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

21/12/21 23:21:09 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.8 instead (on interface wlp59s0)
21/12/21 23:21:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/21 23:21:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/21 23:21:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Airport

In [3]:
# AIRPORT DATAFRAME

airport_schema = StructType([
    StructField('airport', StringType(), True),
    StructField('city', StringType(), True),
    StructField('country', StringType(), True),
    StructField('IATA', StringType(), True),
    StructField('ICAO', StringType(), True),
    StructField('lat', FloatType(), True),
    StructField('lon', FloatType(), True),
    StructField('alt', StringType(), True),
    StructField('TZ', FloatType(), True),
    StructField('dst', StringType(), True),
    StructField('tz2', StringType(), True),
])


airport_df = spark.read.csv(
    "./data/export_luchthavens.txt", 
    # header=True,
    sep="\t",
    multiLine=True,
    schema=airport_schema
)

try:
    airport_df.write.parquet("aws/airport.parquet")
except:
    print("")
airport_df

airport,city,country,IATA,ICAO,lat,lon,alt,TZ,dst,tz2
Airport,City,Country,IATA,ICAO,null,null,Alt,null,DST,Tz
Bamyan Airport,Bamyan,Afghanistan,BIN,OABN,34.816666,67.816666,2550,4.5,N,Asia/Kabul
Camp Bastion,Camp Bastion,Afghanistan,null,OAZI,31.865557,64.195274,2808,4.5,N,Asia/Kabul
Chaghcharan Airport,Chaghcharan,Afghanistan,CCN,OACC,34.52667,65.27167,7383,4.5,U,Asia/Kabul
Faizabad Airport,Faizabad,Afghanistan,FBD,OAFZ,37.1211,70.5181,3872,4.5,U,Asia/Kabul
Herat,Herat,Afghanistan,HEA,OAHR,34.21002,62.2283,3206,4.5,U,Asia/Kabul
Jalalabad,Jalalabad,Afghanistan,JAA,OAJL,34.39984,70.49863,1814,4.5,U,Asia/Kabul
Bagram AFB,Kabul,Afghanistan,BPM,OAIX,34.5646,69.1554,4895,4.5,U,Asia/Kabul
Kabul Intl,Kabul,Afghanistan,KBL,OAKB,34.565853,69.212326,5877,4.5,U,Asia/Kabul
Kandahar,Kandahar,Afghanistan,KDH,OAKN,31.505756,65.847824,3337,4.5,U,Asia/Kabul


## Weather

In [4]:
# WEATHER DATAFRAME

weather_schema = StructType([
    StructField('date', DateType(), True),
    StructField('DDVEC', IntegerType(), True),
    StructField('FHVEC', IntegerType(), True),
    StructField('FG', IntegerType(), True),
    StructField('FHX', IntegerType(), True),
    StructField('FHXH', IntegerType(), True),
    StructField('FHN', IntegerType(), True),
    StructField('FHNH', IntegerType(), True),
    StructField('FXX', IntegerType(), True),
    StructField('FXXH', IntegerType(), True),
    StructField('TG', IntegerType(), True),
    StructField('TN', IntegerType(), True),
    StructField('TNH', IntegerType(), True),
    StructField('TX', IntegerType(), True),
    StructField('TXH', IntegerType(), True),
    StructField('T10N', IntegerType(), True),
    StructField('T10NH', IntegerType(), True),
    StructField('SQ', IntegerType(), True),
    StructField('Q', IntegerType(), True),
    StructField('DR', IntegerType(), True),
    StructField('RH', IntegerType(), True),
    StructField('RHX', IntegerType(), True),
    StructField('RHXH', IntegerType(), True),
    StructField('PG', IntegerType(), True),
    StructField('PX', IntegerType(), True),
    StructField('PXH', IntegerType(), True),
    StructField('PN', IntegerType(), True),
    StructField('PNH', IntegerType(), True),
    StructField('VVN', IntegerType(), True),
    StructField('VVNH', IntegerType(), True),
    StructField('VVX', IntegerType(), True),
    StructField('VVXH', IntegerType(), True),
    StructField('NG', IntegerType(), True),
    StructField('UG', IntegerType(), True),
    StructField('UX', IntegerType(), True),
    StructField('UXH', IntegerType(), True),
    StructField('UN', IntegerType(), True),
    StructField('UNH', IntegerType(), True),
    StructField('EV2', IntegerType(), True),
])

weather_df = spark.read.csv(
    "./data/export_weer.txt", 
    header=True,
    sep="\t",
    multiLine=True,
    schema=weather_schema
)

try:
    weather_df.write.parquet("aws/weather.parquet")
except:
    print("")
weather_df

21/12/21 23:21:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


21/12/21 23:21:14 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 40, schema size: 39
CSV file: file:///home/lenny/Projects/DataProjects/vision-airport/data/export_weer.txt
21/12/21 23:21:15 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 40, schema size: 39
CSV file: file:///home/lenny/Projects/DataProjects/vision-airport/data/export_weer.txt


date,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,FXXH,TG,TN,TNH,TX,TXH,T10N,T10NH,SQ,Q,DR,RH,RHX,RHXH,PG,PX,PXH,PN,PNH,VVN,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,EV2
2014-01-01,24,35,38,60,1,0,23,110,1,-16,-63,24,7,14,-91,24,42,54,318,0,-1,-1,2,10026,10094,24,9978,3,57,23,75,2,4,78,93,23,67,12
2014-01-02,212,22,31,50,18,10,1,80,21,-11,-64,1,12,19,-86,6,0,0,117,8,1,1,17,10140,10172,24,10101,1,37,23,61,5,8,91,94,9,89,14
2014-01-03,33,17,25,60,1,0,18,100,2,-39,-65,21,-10,14,-89,24,62,79,388,0,0,0,1,10220,10238,20,10176,1,50,21,75,14,4,84,92,20,69,13
2014-01-04,199,30,31,50,14,20,1,80,12,-29,-54,8,-6,24,-57,6,10,13,179,0,-1,-1,1,10159,10222,1,10099,24,2,22,57,12,8,90,99,21,83,13
2014-01-05,194,28,28,40,14,20,2,60,9,-7,-30,22,19,13,-40,24,33,42,253,8,9,8,2,10053,10094,1,10030,24,2,3,63,14,7,91,97,1,82,13
2014-01-06,196,24,25,40,19,20,2,60,16,-30,-52,3,-5,13,-75,6,65,82,455,9,1,1,23,10040,10059,23,10027,3,24,23,58,14,5,86,93,3,76,12
2014-01-07,160,16,20,40,12,10,16,60,12,-49,-100,24,-21,14,-123,24,56,70,408,0,-1,-1,1,10110,10175,24,10061,1,4,23,56,13,3,89,99,23,76,13
2014-01-08,43,35,36,60,20,10,1,110,20,-50,-107,2,-21,24,-131,6,29,36,216,0,0,0,1,10216,10232,21,10178,1,18,1,68,24,6,87,93,4,77,21
2014-01-09,32,73,73,80,8,70,1,150,14,-22,-49,3,-5,24,-54,6,0,0,166,58,10,3,16,10196,10228,1,10158,24,23,15,75,4,8,84,90,20,78,13
2014-01-10,51,35,35,60,1,10,16,120,1,-5,-10,21,2,12,-15,24,0,0,149,49,11,3,14,10171,10194,24,10153,5,31,9,70,1,8,87,94,17,82,11


## Customers

In [5]:
# CUSTOMERS DATAFRAME

customers_schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('operation', FloatType(), True),
    StructField('facilities', FloatType(), True),
    StructField('shops', FloatType(), True),
])


customers_df = spark.read.csv(
    "./data/export_klant.csv", 
    header=True,
    sep=";",
    multiLine=True,
    schema=customers_schema
)

try:
    customers_df.write.parquet("aws/customers.parquet")
except:
    print("")
customers_df

21/12/21 23:21:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Vluchtid, Operatie, Faciliteiten, Shops
 Schema: id, operation, facilities, shops
Expected: id but found: Vluchtid
CSV file: file:///home/lenny/Projects/DataProjects/vision-airport/data/export_klant.csv
21/12/21 23:21:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Vluchtid, Operatie, Faciliteiten, Shops
 Schema: id, operation, facilities, shops
Expected: id but found: Vluchtid
CSV file: file:///home/lenny/Projects/DataProjects/vision-airport/data/export_klant.csv


id,operation,facilities,shops
1317307,7.2,8.5,null
1317327,7.4,8.0,null
1317352,7.0,6.9,6.9
1317395,6.5,7.3,7.0
1317406,6.9,6.9,6.3
1317488,6.4,5.2,6.8
1317506,4.7,5.9,7.1
1317508,6.7,8.5,6.2
1317514,6.1,3.6,7.2
1317559,6.0,7.6,6.6


## Vertrek

In [6]:
vertrek_schema = StructType([
    StructField("Vluchtid", IntegerType(), False),
    StructField("Vliegtuigcode",StringType(),True),
    StructField("Terminal", StringType(),True),
    StructField("Gate", StringType(), True),
    StructField("Baan", ShortType(), True),
    StructField("Bezetting", IntegerType(), True),
    StructField("Vracht", IntegerType(), True),
    StructField("Vertrektijd", TimestampType(), True)
  ])

vertrek_df = spark.read.csv(
    DATADIR + "/export_vertrek.txt", 
    header=True,
    sep='\t',
    schema=vertrek_schema
)
try:
    vertrek_df.write.parquet("aws/vertrek.parquet")
except:
    print("")

vertrek_df

Vluchtid,Vliegtuigcode,Terminal,Gate,Baan,Bezetting,Vracht,Vertrektijd
935992,VTK297,B,B5,2,69,null,2014-01-01 02:46:00
935993,VDL5829,D,D5,2,78,null,2014-01-01 02:57:00
935994,VSU1831,D,D3,2,85,null,2014-01-01 03:07:00
935995,VEZY741,C,C4,2,54,null,2014-01-01 03:33:00
935996,VKL499,C,C2,2,71,null,2014-01-01 03:48:00
935997,VGA4943,D,D2,2,69,null,2014-01-01 03:50:00
935999,VKL315,C,C1,2,71,null,2014-01-01 04:04:00
936000,VHV381,A,A4,2,49,null,2014-01-01 04:05:00
936002,VBT361,B,B4,2,86,null,2014-01-01 04:17:00
936003,VDY439,C,C7,2,84,null,2014-01-01 04:19:00


## Aankomst

In [7]:
aankomst_schema = StructType([
    StructField("Vluchtid", IntegerType(), False),
    StructField("Vliegtuigcode",StringType(),True),
    StructField("Terminal", StringType(), True),
    StructField("Gate", StringType(), True),
    StructField("Baan", ShortType(), True),
    StructField("Bezetting", IntegerType(), True),
    StructField("Vracht", IntegerType(), True),
    StructField("Aankomsttijd", TimestampType(), True)
  ])

aankomst_df = spark.read.csv(
    DATADIR + "/export_aankomst.txt", 
    header=True,
    sep='\t',
    schema=aankomst_schema
)

try:
    aankomst_df.write.parquet("aws/aankomst.parquet")
except:
    print("")
aankomst_df

Vluchtid,Vliegtuigcode,Terminal,Gate,Baan,Bezetting,Vracht,Aankomsttijd
935998,VFR735,A,A1,1,45,null,2014-01-01 03:53:00
936001,VHV303,A,A2,1,62,null,2014-01-01 04:15:00
936009,VKL301,C,C1,1,58,null,2014-01-01 04:38:00
936012,VTK505,B,B5,1,67,null,2014-01-01 04:43:00
936013,VKL859,C,C2,1,71,null,2014-01-01 04:54:00
936018,VA3599,A,A6,1,61,null,2014-01-01 05:35:00
936020,VCAO9406,F,null,1,null,33,2014-01-01 05:54:00
936021,VKL447,C,C1,1,82,null,2014-01-01 05:58:00
936022,VKL2761,E,E2,1,70,null,2014-01-01 05:59:00
936023,VFDX9706,F,null,1,null,36,2014-01-01 05:59:00


## Planning

In [8]:
planning_schema = StructType([
    StructField("Vluchtnr", IntegerType(), False),
    StructField("Airlinecode", StringType(), True),
    StructField("Destcode", StringType(), True),
    StructField("Planterminal", StringType(), True),
    StructField("Plangate", StringType(), True),
    StructField("Plantijd", StringType(), True)
  ])

planning_df = spark.read.csv(
    DATADIR + "/export_planning.txt", 
    header=True,
    sep='\t',
    schema=planning_schema
)
try:
    planning_df.write.parquet("aws/planning.parquet")
except:
    print("")
planning_df

Vluchtnr,Airlinecode,Destcode,Planterminal,Plangate,Plantijd
null,9W,DEL,D,D2,2:10 PM
null,9W,DEL,D,D2,3:25 PM
null,A3,ATH,A,A6,5:10 AM
null,A3,ATH,A,A6,5:55 AM
null,A3,ATH,A,A6,2:45 PM
null,A3,ATH,A,A6,3:30 PM
null,A3,ATH,A,A6,8:05 AM
null,A3,ATH,A,A6,8:50 AM
null,A3,ATH,A,A6,8:50 PM
null,A3,ATH,A,A6,9:35 PM


## Banen

In [9]:

banen_schema = StructType([
    StructField("Baannummer", IntegerType(), True),
    StructField("Code",StringType(),True),
    StructField("Naam", StringType(),True),
    StructField("Lengte", IntegerType(), True),
  ])

banen_df = spark.read.csv(
    "./data/export_banen.csv", 
    header=True,
    sep=';',
    schema= banen_schema
)

try:
    banen_df.write.parquet("aws/banen.parquet")
except:
    print("")
banen_df

Baannummer,Code,Naam,Lengte
1,09-27,Marloes Boumansbaan,3200
2,18R-36L,Joost van den Bem...,3600
3,18L-36R,Frank de Ruijterbaan,4000
4,18C-36C,Stépan Breedveldbaan,3500
5,06-24,Hans Dellebekebaan,3500
6,04-22,Sjoerd Eversbaan,2200


## Maatschappijen

In [10]:
maatschappijen_schema = StructType([
    StructField("Name", StringType(),True),
    StructField("IATA", StringType(), True),
    StructField("ICAO",StringType(),True),
  ])

maatschappijen_df = spark.read.csv(
    "./data/export_maatschappijen.txt", 
    header=True,
    sep='\t',
    schema=maatschappijen_schema
)

try:
    maatschappijen_df.write.parquet("aws/maatschappijen.parquet")
except:
    print("")
maatschappijen_df

Name,IATA,ICAO
Private flight,-,N/A
Unknown,-,N/A
ELK Airways,--,ELK
Tom\\'s & co airl...,&T,T&O
Wilderness Air,;;,\N
Tyrolean Airways,\N,TYR
Whitaker Air,^^,\N
U.S. Air,-+,--+
Bobb Air Freight,1,\N
Blue Air,0B,JOR


## Vlucht

In [11]:
vlucht_schema = StructType([
    StructField("Vluchtid", IntegerType(), True),
    StructField("Vluchtnr",StringType(),True),
    StructField("Airlinecode", StringType(),True),
    StructField("Destcode", StringType(), True),
    StructField("Vliegtuigcode", StringType(), True),
    StructField("Datum", DateType(), True)
  ])

vlucht_df = spark.read.csv(
    "./data/export_vlucht.txt", 
    header=True,
    sep='\t',
    schema=vlucht_schema
)

try:
    vlucht_df.write.parquet("aws/vlucht.parquet")
except:
    print("")
vlucht_df


Vluchtid,Vluchtnr,Airlinecode,Destcode,Vliegtuigcode,Datum
935992,TK297,TK,ESB,VTK297,2014-01-01
935993,DL5829,DL,DTW,VDL5829,2014-01-01
935994,SU1831,SU,SVO,VSU1831,2014-01-01
935995,EZY741,EZY,ACE,VEZY741,2014-01-01
935996,KL500,KL,VIE,VKL499,2014-01-01
935997,GA4943,GA,CGK,VGA4943,2014-01-01
935998,FR734,FR,AYT,VFR735,2014-01-01
935999,KL316,KL,IST,VKL315,2014-01-01
936000,HV382,HV,FAO,VHV381,2014-01-01
936001,HV303,HV,ATH,VHV303,2014-01-01


## Vliegtuig

In [12]:
vliegtuig_schema = StructType([
    StructField("Airlinecode", StringType(),True),
    StructField("Vliegtuigcode", StringType(), True),
    StructField("Vliegtuigtype",StringType(),True),
    StructField("Bouwjaar", IntegerType(), True)
  ])

vliegtuig_df = spark.read.csv(
    "./data/export_vliegtuig.txt", 
    header=True,
    sep='\t',
    schema=vliegtuig_schema
)

try:
    vliegtuig_df.write.parquet("aws/vliegtuig.parquet")
except:
    print("")
vliegtuig_df

Airlinecode,Vliegtuigcode,Vliegtuigtype,Bouwjaar
TK,VTK297,321,1970
DL,VDL5829,320,2007
SU,VSU1831,321,2012
EZY,VEZY741,738,1978
KL,VKL499,738,1982
GA,VGA4943,752,1982
FR,VFR735,320,1984
KL,VKL315,73G,1971
HV,VHV381,772,2003
HV,VHV303,73G,2009


## Vliegtuigtype

In [13]:
#VliegtuigType

vliegtuig_type_schema = StructType([
    StructField("IATA", StringType(), True),
    StructField("ICAO",StringType(),True),
    StructField("Merk", StringType(),True),
    StructField("Type", StringType(), True),
    StructField("Wake", StringType(), True),
    StructField("Cat", StringType(), True),
    StructField("Capaciteit", IntegerType(), True),
    StructField("Vracht", IntegerType(), True)
  ])

vliegtuig_type_df = spark.read.csv(
    "./data/export_vliegtuigtype.csv", 
    header=True,
    sep=';',
    schema=vliegtuig_type_schema
)

try:
    vliegtuig_type_df.write.parquet("aws/vliegtuig_type.parquet")
except:
    print("")
vliegtuig_type_df

IATA,ICAO,Merk,Type,Wake,Cat,Capaciteit,Vracht
100,F100,Fokker,Fokker 100,M,Pax,100,2
141,B461,British Aerospace,BAe 146-100 Pax,M,Pax,105,5
142,B462,British Aerospace,BAe 146-200 Pax,M,Pax,115,5
143,B463,British Aerospace,BAe 146-300 Pax,M,Pax,130,5
146,null,British Aerospace,BAe 146 all pax m...,M,Pax,null,null
310,A310,Airbus,Airbus A310 all p...,H,Pax,250,5
312,A310,Airbus,Airbus A310-200 pax,H,Pax,250,5
313,A310,Airbus,Airbus A310-300 pax,H,Pax,250,5
318,A318,Airbus,Airbus A318,M,Pax,105,8
319,A319,Airbus,Airbus A319,M,Pax,125,8


# Tensorflow

In [14]:
facts = pd.read_csv("./data/vertrek_facts.csv")

facts = facts.dropna(subset=['operation'])
facts['operation'] = facts['operation'].apply(lambda x: x*10)
facts = facts.drop(columns=['vracht', 'shops', 'facilities', 'vertrektijd', 'plantijd', 'plantijd','name', 'datum','vliegtuigcode', 'vluchtid', 'gate', 'plangate', 'terminal', 'planterminal', 'airlinecode', 'naam', 'windsnelheid_gem', 'temp_gem', 'neerslag_som', 'opgetreden_zicht_max', 'vluchtnr', 'dst']).dropna(subset=['vracht_vliegtuigtype'])

facts.head()

,baan,bezetting,operation,code,lengte,destcode,vliegtuigtype,bouwjaar,merk,type,...,wake,cat,capaciteit,airport,city,country,lat,lon,alt,vertraging_min
187,3,61,76.0,18L-36R,4000,JFK,763,1998,Boeing,Boeing 767-300 pax,...,H,Pax,250,John F Kennedy Intl,New York,United States,40.63975,-73.77892,13,58.0
258,1,66,66.0,09-27,3200,JFK,763,1998,Boeing,Boeing 767-300 pax,...,H,Pax,250,John F Kennedy Intl,New York,United States,40.63975,-73.77892,13,57.0
349,1,64,70.0,09-27,3200,JFK,763,1998,Boeing,Boeing 767-300 pax,...,H,Pax,250,John F Kennedy Intl,New York,United States,40.63975,-73.77892,13,73.0
375,3,67,76.0,18L-36R,4000,JFK,763,1998,Boeing,Boeing 767-300 pax,...,H,Pax,250,John F Kennedy Intl,New York,United States,40.63975,-73.77892,13,54.0
610,3,92,72.0,18L-36R,4000,JFK,763,1998,Boeing,Boeing 767-300 pax,...,H,Pax,250,John F Kennedy Intl,New York,United States,40.63975,-73.77892,13,77.0


In [15]:
feats = ['merk', 'type', 'wake', 'cat','code', 'vliegtuigtype','type', 'destcode','airport', 'country', 'city']
facts_final = pd.get_dummies(facts,columns=feats,drop_first=True)

x = facts_final.drop(['operation'],axis=1).values
y = facts_final['operation'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [16]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [17]:
model = Sequential()
model.add(Dense(800, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(400, activation = "relu"))
model.add(Dense(200, activation = "relu"))
model.add(Dropout(.1))
model.add(Dense(100))

model.compile(optimizer= "adam",loss = "mean_squared_error",metrics = ["accuracy"])

model.fit(x_train, y_train, epochs = 15)

2021-12-21 23:21:18.456750: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 23:21:18.460156: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-12-21 23:21:18.460884: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-12-21 23:21:18.461463: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library

Epoch 1/15
44/44 [==============================] - 0s 4ms/step - loss: 1710.4712 - accuracy: 0.0142
Epoch 2/15
44/44 [==============================] - 0s 3ms/step - loss: 144.0029 - accuracy: 0.0171
Epoch 3/15
44/44 [==============================] - 0s 3ms/step - loss: 96.5561 - accuracy: 0.0078
Epoch 4/15
44/44 [==============================] - 0s 3ms/step - loss: 90.7780 - accuracy: 0.0192
Epoch 5/15
44/44 [==============================] - 0s 3ms/step - loss: 83.4502 - accuracy: 0.0128
Epoch 6/15
44/44 [==============================] - 0s 4ms/step - loss: 80.7364 - accuracy: 0.0100
Epoch 7/15
44/44 [==============================] - 0s 3ms/step - loss: 77.5318 - accuracy: 0.0114
Epoch 8/15
44/44 [==============================] - 0s 3ms/step - loss: 78.5654 - accuracy: 0.0142
Epoch 9/15
44/44 [==============================] - 0s 4ms/step - loss: 74.2134 - accuracy: 0.0092
Epoch 10/15
44/44 [==============================] - 0s 4ms/step - loss: 72.0870 - accuracy: 0.0078
Epoch 

In [18]:
facts["operation"].iloc[1]/10

6.6

In [19]:
prediction = model.predict(x_train)
guess = np.argmax(prediction[1])
print(guess/10)

4.0


In [20]:
print(f'Merk:  {stats.spearmanr(facts["merk"], facts["operation"])[0]}')
print(f'Type: {stats.spearmanr(facts["type"], facts["operation"])[0]}')
print(f'Cat: {stats.spearmanr(facts["cat"], facts["operation"])[0]}')
print(f'Code: {stats.spearmanr(facts["code"], facts["operation"])[0]}')
print(f'Destcode: {stats.spearmanr(facts["destcode"], facts["operation"])[0]}')
print(f'Airport: {stats.spearmanr(facts["airport"], facts["operation"])[0]}')
print(f'Country: {stats.spearmanr(facts["country"], facts["operation"])[0]}')
print(f'City: {stats.spearmanr(facts["city"], facts["operation"])[0]}')
print(f'Bezetting: {stats.spearmanr(facts["bezetting"], facts["operation"])[0]}')
print(f'Lengte: {stats.spearmanr(facts["lengte"], facts["operation"])[0]}')
print(f'Baan: {stats.spearmanr(facts["baan"], facts["operation"])[0]}')
print(f'Bouwjaar: {stats.spearmanr(facts["bouwjaar"], facts["operation"])[0]}')
print(f'Lat: {stats.spearmanr(facts["lat"], facts["operation"])[0]}')
print(f'Lon: {stats.spearmanr(facts["lon"], facts["operation"])[0]}')
print(f'Alt: {stats.spearmanr(facts["alt"], facts["operation"])[0]}')
print(f'Vertraging in minuten: {stats.spearmanr(facts["vertraging_min"], facts["operation"])[0]}')

Merk:  -0.013015843728183506
Type: -0.03316365001409106
Cat: nan
Code: 0.04654339294085504
Destcode: -0.02973566265508927
Airport: -0.02005505872328328
Country: 0.00952505170318679
City: -0.02813864735010377
Bezetting: -0.0428534895047835
Lengte: 0.09972965566596169
Baan: -0.004949962709032542
Bouwjaar: -0.044185068188028406
Lat: 0.0570387511071401
Lon: -0.015722021437659477
Alt: 0.04598339622852409
Vertraging in minuten: -0.282294634094083


# Conclusie

Uit de verschillende spearman correlatiewaarden kunnen we lezen dat vertraging veruit de grootste impact heeft op de opinie van de klanten.